# 1. Подготовка

Импортируем библиотеки.

In [1]:
import requests
import pathlib
import client_lib # импортируем библиотеку для работы с ML Space

Устанавливаем переменные.

In [2]:
BASE_DIR = str(pathlib.Path().absolute())
print(f"Working dir: {BASE_DIR}")

Working dir: /home/jovyan/aicloud-examples/quick-start/job_launch_cpu


# 2. Запуск задачи обучения

Запускать задачу в регионе будем с помощью класса `client_lib.Job()`.

Обязательные параметры для запуска задачи обучения:
- `script` – путь к запускаемому скрипту;
- `base_image` – базовый образ, в котором будет исполняться скрипт обучения модели;
- `instance_type` – конфигурация вычислительных ресурсов, используемых для решения задач.

Подробное описание параметров – в [документации client_lib](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/client-lib__job.html).

По умолчанию задачи запускаются в регионе Christofari.V100. Чтобы запустить задачу в другом регионе, необходимо указать его в параметре `region`. 

См. [Список регионов и их обозначения в client_lib](https://cloud.ru/docs/aicloud/mlspace/concepts/client-lib__job.html#row-body-region).

Для примера запустим задачу в регионе Cloud.Region.A100, обозначенном как `SR002-MT`.

Для масштабирования задачи доступны следующие параметры:

- `n_workers` – количество рабочих узлов региона, на котором будет исполняться скрипт;
- `instance_type` – конфигурация вычислительных ресурсов, используемых для решения задач.

Для выбора значения параметра `instance_type` воспользуемся методом `get_instance_types()`.

Выведем доступные конфигурациидля региона `SR003`.

In [3]:
client_lib.get_instance_types(regions="SR003")

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━┳━━━━━┓
┃ region ┃ instance_type                ┃ memory ┃ cpu ┃ gpu ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━╇━━━━━┩
│ SR003  │ a100.1gpu                    │ 243Gi  │ 16  │ 1   │
│ SR003  │ a100.2gpu                    │ 486Gi  │ 32  │ 2   │
│ SR003  │ a100.3gpu                    │ 729Gi  │ 48  │ 3   │
│ SR003  │ a100.4gpu                    │ 972Gi  │ 64  │ 4   │
│ SR003  │ a100.5gpu                    │ 1215Gi │ 80  │ 5   │
│ SR003  │ a100.6gpu                    │ 1458Gi │ 96  │ 6   │
│ SR003  │ a100.7gpu                    │ 1701Gi │ 112 │ 7   │
│ SR003  │ a100.8gpu                    │ 1944Gi │ 128 │ 8   │
│ SR003  │ a100.1gpu.8C.243G            │ 243Gi  │ 8   │ 1   │
│ SR003  │ a100.2gpu.16C.486G           │ 486Gi  │ 16  │ 2   │
│ SR003  │ a100.3gpu.24C.729G           │ 729Gi  │ 24  │ 3   │
│ SR003  │ a100.4gpu.32C.972G           │ 972Gi  │ 32  │ 4   │
│ SR003  │ a100.5gpu.40C.1215G          │ 1215Gi │ 40  │ 5   │
│ SR003  │ a100.6gpu.48C.1458G          │ 1458Gi │ 48  │ 6   │
│ SR003  │ a100.7gpu.56C.1701G          │ 1701Gi │ 56  │ 7   │
│ SR003  │ a100.8gpu.64C.1944G          │ 1944Gi │ 64  │ 8   │
│ SR003  │ a100plus.1gpu.80vG.12C.182G  │ 182Gi  │ 12  │ 1   │
│ SR003  │ a100plus.2gpu.80vG.24C.364G  │ 364Gi  │ 24  │ 2   │
│ SR003  │ a100plus.3gpu.80vG.36C.546G  │ 546Gi  │ 36  │ 3   │
│ SR003  │ a100plus.4gpu.80vG.48C.728G  │ 728Gi  │ 48  │ 4   │
│ SR003  │ a100plus.5gpu.80vG.60C.910G  │ 910Gi  │ 60  │ 5   │
│ SR003  │ a100plus.6gpu.80vG.72C.1092G │ 1092Gi │ 72  │ 6   │
│ SR003  │ a100plus.7gpu.80vG.84C.1274G │ 1274Gi │ 84  │ 7   │
│ SR003  │ a100plus.8gpu.80vG.96C.1456G │ 1456Gi │ 96  │ 8   │
│ SR003  │ cpu.2C.8G                    │ 8Gi    │ 2   │ 0   │
│ SR003  │ cpu.8C.32G                   │ 32Gi   │ 8   │ 0   │
│ SR003  │ cpu.16C.64G                  │ 64Gi   │ 16  │ 0   │
│ SR003  │ cpu.12C.48G                  │ 48Gi   │ 12  │ 0   │
│ SR003  │ cpu-ai-3xgiant               │ 192Gi  │ 48  │ 0   │
│ SR003  │ cpu-ai-2xgiant               │ 128Gi  │ 32  │ 0   │
└────────┴──────────────────────────────┴────────┴─────┴─────┘

Для примера запустим задачу на 1 воркере с конфигурацией 32GB RAM и 8 CPU.

Сохраним в переменные название региона, `instanse_type` и образа.

In [33]:
REGION = "SR003"
INSTANCE_TYPE = "cpu.8C.32G"
N_WORKERS = 1
BASE_IMAGE = 'cr.ai.cloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0'

In [34]:
job = client_lib.Job(
    base_image=BASE_IMAGE,
    script=f'{BASE_DIR}/xgboost-boston-house-price.py',
    region=REGION,
    instance_type=INSTANCE_TYPE,
    n_workers=N_WORKERS,
    type="pytorch2",
    processes_per_worker=1,
    job_desc="pytorch2 | client_lib | use_env=False | CPU-job",
)


Запустим задачу методом `submit()`.

In [35]:
job.submit()

'Job "lm-mpi-job-172804b7-72c5-40fd-ba76-29fe96648060" created.'

Для получения статуса задачи воспользуемся методом `status()`.

См. [Список статусов задачи](https://cloud.ru/docs/aicloud/mlspace/concepts/guides/guides__env__common/environments__model-training__overview-statistics.html).

In [37]:
job.status()

'Job status=Running'

Для просмотра логов задачи вызовем метод `logs()`.

Логи будут доступны после запуска задачи, то есть ее перехода в статус "Running".

In [38]:
job.logs()

2024-05-24T13:30:12.655223156Z Job lm-mpi-job-172804b7-72c5-40fd-ba76-29fe96648060 is starting...
2024-05-24T13:30:12.726321310Z 🕒 Waiting for workers to be ready... 🕒
2024-05-24T13:30:27.738773286Z Connecting to mpimaster-0 ..... Ready ✓
2024-05-24T13:30:27.738798875Z 🚀 All workers are READY 🚀
2024-05-24T13:30:28.929731014Z [1,0]<stdout>:Working dir: /home/jovyan
2024-05-24T13:30:28.944478031Z [1,0]<stdout>:<class 'pandas.core.frame.DataFrame'>
2024-05-24T13:30:28.944509359Z [1,0]<stdout>:RangeIndex: 20640 entries, 0 to 20639
2024-05-24T13:30:28.944513341Z [1,0]<stdout>:Data columns (total 9 columns):
2024-05-24T13:30:28.944515980Z [1,0]<stdout>: #   Column      Non-Null Count  Dtype  
2024-05-24T13:30:28.944518558Z [1,0]<stdout>:---  ------      --------------  -----  
2024-05-24T13:30:28.944521264Z [1,0]<stdout>: 0   MedInc      20640 non-null  float64
2024-05-24T13:30:28.944523764Z [1,0]<stdout>: 1   HouseAge    20640 non-null  float64
2024-05-24T13:30:28.944526440Z [1,0]<stdout>: 

Задача завершится автоматически после выполнения скрипта. Если необходимо прервать выполнение задачи, используется методом `kill()`.

In [32]:
job.kill()

'Job "lm-mpi-job-7a81772c-1c88-4437-ad70-119c68f791cb" deleted'

# 3. [Опционально] Сохранение промежуточных результатов обучения модели

Если в процессе обучения модели вы сохраняете промежуточные результаты, или чекпоинты, обучения, они попадают в папку `./logs`. Их можно скачать через веб-интерфейс Jupyter-ноутбука или скопировать из локально доступной файловой системы в хранилище S3.

Внутри кода задачи обучения можно сохранять метрики модели с помощью MLflow. Пример — в коде исполняемого файла `xgboost-boston-house-price.py`.